In [31]:
# ---- Enable importing from code/ folder ----
import sys
sys.path.append("code")

# ---- Import project utilities ----
from utils import (
    load_series,
    load_fred,
    monthly_wide_to_quarterly,
    load_simple_series,
    pct_change,
    log_diff,
    to_quarterly,
    to_monthly,
    load_for_arimax
)

print("utils.py loaded successfully!")


utils.py loaded successfully!


In [ ]:
def monthly_to_quarterly(path, value_name):
    df = pd.read_csv(path)

    # Melt wide to long
    df_long = df.melt(id_vars="Year", var_name="Month", value_name=value_name)

    # Clean blank cells
    df_long[value_name] = df_long[value_name].replace(" ", pd.NA)

    # Convert to numeric
    df_long[value_name] = pd.to_numeric(df_long[value_name], errors="coerce")

    # Drop missing rows
    df_long = df_long.dropna(subset=[value_name])

    # Convert Month names → month numbers
    df_long["Month"] = pd.to_datetime(df_long["Month"], format="%b").dt.month

    # Build date column
    df_long["date"] = pd.to_datetime(
        df_long[["Year", "Month"]].assign(day=1)
    )

    # Set index
    df_long = df_long.set_index("date").sort_index()

    monthly = df_long[[value_name]]

    # Convert to quarterly mean
    quarterly = monthly.resample("QE").mean()

    return quarterly


In [38]:
ppi = load_series("../data/ppi.csv", value_name="ppi")
ppi


,ppi
date,
1913-01-01,12.100
1913-02-01,12.000
1913-03-01,12.000
1913-04-01,12.000
1913-05-01,11.900
...,...
2025-04-01,258.392
2025-05-01,258.642
2025-06-01,260.690


In [24]:
def read_fred_monthly(path, value_col=None, dropna=True):

    df = pd.read_csv(path)

    # Identify the value column if not provided
    if value_col is None:
        value_col = df.columns[1]  # FRED always DATE + VALUE

    # Convert dates
    df["DATE"] = pd.to_datetime(df["DATE"])

    # Convert values to numeric
    df[value_col] = pd.to_numeric(df[value_col], errors="coerce")

    # Drop NaN if desired
    if dropna:
        df = df.dropna(subset=[value_col])

    # Set index
    df = df.set_index("DATE").sort_index()

    # Monthly → Quarterly (average)
    quarterly = df[value_col].resample("QE").mean().to_frame()

    return quarterly


In [29]:
export_q = monthly_to_quarterly("../data/export_index.csv", "Export")
print(export_q.head())
import_q = monthly_to_quarterly("../data/import_index.csv", "Import")
print(import_q.head())
cpi_q = read_fred_monthly("../data/cpi.csv")
ppi_q = read_fred_monthly("../data/ppi.csv")

ppi_q


            Export
date              
1984-03-31    85.9
1984-06-30    87.1
1984-09-30    85.1
1984-12-31    84.2
1985-03-31    84.0
            Import
date              
1984-03-31    78.7
1984-06-30    78.9
1984-09-30    77.7
1984-12-31    77.0
1985-03-31    75.5


,PPIACO
DATE,
1913-03-31,12.033333
1913-06-30,11.933333
1913-09-30,12.066667
1913-12-31,12.066667
1914-03-31,11.766667
...,...
2024-09-30,255.155333
2024-12-31,253.238333
2025-03-31,258.461000
